In [ ]:
import tifffile
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os
import pandas as pd
import sys
sys.path.append(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\10_code\UTvsXCT-preprocessing')
import dbtools as db
from dbtools import load as load
from preprocess_tools import io, sample_isolater

# Database conection

In [ ]:
try:
    conn = db.connect()
    print("Connected to the database")
except Exception as error:
    print(error)

# Data retrieval

We have to load the data from the database to get:

1. The file ids to use them as parent measurement

## Measurements file paths

In [ ]:
original_paths = [Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Fabricacion Nacho\05_Probetas_Nacho_2025\medidas\Na_01_1_2_3_4\volume_eq')]

measurements_table = db.relation_metadata('measurements','samples','sample_measurements')

parent_id_column = 'measurementtype_id_measurement'

saving_folder = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Fabricacion Nacho\05_Probetas_Nacho_2025\probetas')

# for each original path, get the id_measurement where file_path_measurement is equal to the str of the original path

original_ids = []
original_measurementtype_ids = []
original_sample_names = []

for original_path in original_paths:
    original_path_str = str(original_path)
    original_id = measurements_table.loc[measurements_table['file_path_measurement'] == original_path_str, 'id_measurement'].values
    original_measurementtype_id = measurements_table.loc[measurements_table['file_path_measurement'] == original_path_str, parent_id_column].values[0]
    original_sample_name = measurements_table.loc[measurements_table['file_path_measurement'] == original_path_str, 'name_sample'].values
    original_ids.append(original_id)
    original_measurementtype_ids.append(original_measurementtype_id)
    original_sample_names.append(original_sample_name)
    print(f"Original path: {original_path_str}, ID: {original_id}", 
          f"Measurement type ID: {original_measurementtype_id}",
          f"Sample names: {original_sample_name}")

## Functions

## Database loading

## main loop

For each file:

1. Load it

2. Separate the samples in it

3. Save each isolated sample

4. Save them to the database

In [ ]:
for i in range(len(original_paths)):
    # Get the original path and ID
    original_path = original_paths[i]
    original_id = original_ids[i]
    original_measurementtype_id = int(original_measurementtype_ids[i])
    measurement_sample_names = original_sample_names[i]
    number_of_samples = len(original_sample_name)

    #load the volume
    volume = io.load_tif(original_path)

    #isolate the samples
    isolated_volumes = sample_isolater.isolate_samples(volume, number_of_samples)

    for j,isolated_volume in enumerate(isolated_volumes):

        folder_name = saving_folder / original_sample_name[j] / 'volume_eq'

        #create folder
        folder_name.mkdir(parents=True, exist_ok=True)

        print(f"Saving isolated volume for sample {original_sample_name[j]} in folder {folder_name}")

        #save the isolated volume
        io.save_tif(folder_name, isolated_volume)

        height = isolated_volume.shape[0]
        width = isolated_volume.shape[1]
        depth = isolated_volume.shape[2]
        dtype = str(isolated_volume.dtype)
        file_type = 'folder'
        aligned = False
        equalized = True
        axes = ['x','y','z']

        transformation = 'Isolated from a volume conatining more samples using sample isolater from preprocess_tools 0.1.22'

        #save in the database
        load.load_xct_measurement(conn=conn,file_path=str(folder_name),measurementtype_id=original_measurementtype_id,height=height,width=width,depth=depth,dtype=dtype,file_type=file_type,sample_names=list(measurement_sample_names),aligned=aligned,equalized=equalized,axes_order=axes,parent_measurement_path=str(original_path),transformations=transformation)

In [ ]:
for j,isolated_volume in enumerate(isolated_volumes):

    folder_name = saving_folder / original_sample_name[j] / 'volume_eq'

    #create folder
    folder_name.mkdir(parents=True, exist_ok=True)

    print(f"Saving isolated volume for sample {original_sample_name[j]} in folder {folder_name}")

    #save the isolated volume
    io.save_tif(folder_name, isolated_volume)

    height = isolated_volume.shape[0]
    width = isolated_volume.shape[1]
    depth = isolated_volume.shape[2]
    dtype = str(isolated_volume.dtype)
    file_type = 'folder'
    aligned = False
    equalized = True
    axes = ['x','y','z']

    transformation = 'Isolated from a volume conatining more samples using sample isolater from preprocess_tools 0.1.22'

    #save in the database
    load.load_xct_measurement(conn=conn,file_path=str(folder_name),measurementtype_id=original_measurementtype_id,height=height,width=width,depth=depth,dtype=dtype,file_type=file_type,sample_names=list(measurement_sample_names),aligned=aligned,equalized=equalized,axes_order=axes,parent_measurement_path=str(original_path),transformations=transformation)